In [10]:
from PreProcessing.Opentext import get_EBS
from jamo import h2j, j2hcj, j2h
from PreProcessing.Dictionary import sd, ad, kopro

In [12]:
import tensorflow as tf
import numpy as np
import os
import time
import re

In [3]:
all_EBS = get_EBS(1,26)

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [4]:
def decimalnumber_to_korean(m):
    answer = ''
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

def remove_newlines(s):
    
    return re.sub(r'[\n]+', ' ', s)

def remove_puncuations(s):
    
    return re.sub(r'[\.?,‘’]+', ' ', s)

In [5]:
def insert_dot(text, dot):
    pattern1 = re.compile(r'(ㅂㄴㅣㄷㅏ)$')   #ㅂ 니다
    pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
    pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
    pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
        elif pattern2.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _2 += 1
            #list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
        elif pattern4.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _4 += 1
        elif pattern5.findall(new_):
            text_list.append(_.replace(_,_+dot))
            _5 += 1
        else:
            text_list.append(_)
            _6 += 1
            #list6.append(_)
    print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    

In [60]:
# EBS 전처리
def EBS_preprocessing(EBS):

    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)
    
#     특수문자 제거
    ebs_punc_removed = remove_puncuations(ebs_alphabet_conversion)
    
#     점 집어넣기
    ebs_dot_inserted = ' '.join(insert_dot(ebs_punc_removed,'.'))

#     띄어쓰기 제거
    #ebs_whitespace_removed = re.sub('[\s]+', '', ebs_dot_inserted)

    ebs_sep_by_lec = ebs_dot_inserted.replace('  ',' ')
    return ebs_sep_by_lec.split('.')

In [7]:
# GOOGLE 전처리
def GOOGLE_preprocessing(GOOGLE):
        
#     개행 삭제
    #google_newline_removed = remove_newlines(GOOGLE)
    
#     수학기호 한국어로 변환
    google_math_conversion = mathsymbol_to_korean(google_newline_removed)
    
#     숫자 한국어로 변환(GOOGLE)
    google_number_conversion = number_to_korean(google_math_conversion)
        
#     알파벳 한국어로 변환(GOOGLE)
    google_alphabet_conversion = alphabet_to_korean(google_number_conversion)
    
#     특수문자 제거
    google_punc_removed = remove_puncuations(google_alphabet_conversion)
    
#     점 집어넣기
    google_dot_inserted = ' '.join(insert_dot(google_punc_removed,'.'))
    
#     띄어쓰기 제거
    #google_whitespace_removed = re.sub('[\s]+', '', google_dot_inserted)

    return google_whitespace_removed

In [61]:
ko_pre_EBS = EBS_preprocessing(all_EBS)

pattern1 = 1872, pattern2= 2455, pattern3 = 0, pattern5 = 781, pattern6 = 1128


In [62]:
ko_pre_EBS[200:400]

[' 선생님이 앞으로 항상 내 주는 그런 숙제라고 생각하시면 돼요 유제 일번부터 보도록 할 텐데',
 ' 직각삼각형 에이비씨에서 코사인 씨를 구하래요',
 ' 결론부터 보고 우리가 거꾸로 가 볼게요',
 ' 코사인은 뭐 분의 뭐 빗변 분의 밑변 그럼 여기서 직각에서 마주보는 변이 빗변이고 밑변은 마주보는 게 높이겠죠 기준각에서 그러니까 자동으로 이 변이 밑변이야 그러면 코사인 씨라는 건 빗변 분의 밑변이니 비씨분의에이씨를 하라는 얘기고 비씨는 몰라 그리고 에이씨는 얼마야 삼루트오야 그러니까 이 문제는 결국 누굴 구하라는 얘기야 비씨를 구하라는 얘기예요 코사인 씨 구하라는 게 결국 비씨 구해서 해결해라 이 얘기죠 그럼 비씨를 구할 수 있는 방법은 뭡니까',
 ' 직각삼각형이니까',
 ' 당연히 피타고라스의 정리이죠 그럼 비씨는 어떻게 된다 빗변이네 루트 나머지 두 변의 제곱의 합이죠 육의제곱더하기삼루트오의제곱 해 주면 구곱하기오는사십오이니까',
 ' 루트팔십일이 돼서 얼마임을 알 수 있어 비씨가 구가 돼요 이 길이가 구이구나 그렇다면 비씨 자리에 구를 대입해서 구분의삼루트오이니까',
 ' 정답은 삼분의루트오이다 이렇게 여러분들이 유제 일번 해결할 수 있겠다 조금 부지런히 유제 이번 문제도 들어가 보도록 하겠습니다',
 ' 이제 기준각이 좀 왔다 갔다 하면서 여러분들이 좀 어려워집니다',
 ' 그림에서 선분비씨는 원의 지름이에요',
 ' 원의 지름  원에 내접하는 삼각형이 있구나 그러니까 당연히 직각삼각형이다 라는 건 기본 개념으로 알 수 있어야 되겠고 에이비의 길이가 십이이고 비오의 길이가 십이입니다',
 ' 결론부터 볼게요',
 ' 구하라는 게 뭐예요 지금 탄젠트 비입니다',
 ' 탄젠트는 뭐였더라 밑변 분의 높이였었죠 기준각이 누구예요 지금 비 마주보는 게 높이겠네 그렇죠 그러면 이게 빗변이니까',
 ' 밑변이 바로 이 길이가 되는구나 라는 걸 여러분들이 알 수 있으니탄젠트 비를 구하라는 얘기는 결국 밑변 분의 높이  십이분의에이씨를 하라 이 얘기야 그렇죠 

In [ ]:
#ko_pre_GOOGLE = GOOGLE_preprocessing(all_GOOGLE)

In [48]:
!nvidia-smi

Mon Aug 24 19:12:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.23       Driver Version: 442.23       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   41C    P8     1W /  N/A |    460MiB /  4096MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [65]:
#word embedding KOR2VEC

In [75]:
!pip install kor2vec

  Using cached kor2vec-0.0.1a0-py3-none-any.whl (22 kB)


ERROR: Could not find a version that satisfies the requirement torch>=0.4.0 (from kor2vec) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch>=0.4.0 (from kor2vec)


In [73]:
!pip install git+https://github.com/naver/kor2vec.git

  Cloning https://github.com/naver/kor2vec.git to c:\users\kcy93\appdata\local\temp\pip-req-build-l4bhircj


  Running command git clone -q https://github.com/naver/kor2vec.git 'C:\Users\kcy93\AppData\Local\Temp\pip-req-build-l4bhircj'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/naver/kor2vec.git 'C:\Users\kcy93\AppData\Local\Temp\pip-req-build-l4bhircj'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [77]:
!pip install sentencepiece

In [80]:
!git clone https://github.com/google-research/bert

'git' is not recognized as an internal or external command,
operable program or batch file.


In [84]:
import os
form bert

In [87]:

import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
#from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

#sys.path.append(os.pardir)

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder


AttributeError: module 'tensorflow._api.v2.train' has no attribute 'Optimizer'